In [1]:
from keras.applications.nasnet import NASNetMobile
from keras.applications.vgg16 import VGG16
from keras import Model
from keras.layers import Conv2D,MaxPool2D,Input

Using TensorFlow backend.


In [2]:
input_shape = (224,224,3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
def conv2d_block(input_tensor,filters,pooling=True):
    x = Conv2D(filters=filters,kernel_size=(3,3),padding='same',activation='relu')(input_tensor)
    x = Conv2D(filters=filters,kernel_size=(3,3),padding='same',activation='relu')(x)
    if pooling:
        x = MaxPool2D((2,2),strides=(2,2))(x)
    return x

In [4]:
def rpn(input_tensor,anchor):
    x = Conv2D(512,(3,3),padding='same',activation='relu',kernel_initializer='normal')(input_tensor)
    cls = Conv2D(anchor*2,(1,1),activation='sigmoid',kernel_initializer='uniform',name='class_layer')(x)
    rgs = Conv2D(anchor*4,(1,1),activation='linear',kernel_initializer='zero',name='bbox_regressor_layer')(x)
    return [cls,rgs]

In [5]:
inputs = Input(shape=input_shape)
net = conv2d_block(inputs,64)
net = conv2d_block(net,128)
net = conv2d_block(net,256)
net = conv2d_block(net,512,pooling=False)
net = rpn(net,9)
model = Model(inputs = inputs,outputs = net)

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (